### Operations

- General
    - crop/merge (in one command)
    - warp
    - rasterize


- DEMs
    - color relief
    - hill shading
    - texture shading
    - various blends of color relief and shaded relief


- Landsat
    - stacking bands
    - autogain/white-balance
    - pansharpening


__Raw dataset types__
- A set of DEM tiles (NED13 tiles, elev48i100, etc)
- A set of Landsat scenes
- a shapefile


__Basal derived dataset types__
- a cropped and/or merged DEM
- a set of cropped and/or merged Landsat bands
- a rasterized shapefile


A basal derived dataset is necessarily the result of a single crop/merge of a raw dataset. The parameters of this single operation, combined with a raw dataset, define the derived dataset.

Not yet clear how to fit rasterizing shapefiles into this schema. 

### Landsat workflow
1. Crop and/or merge each band
2. Stack three of the merged bands
3. autogain/pansharpen/etc the resulting RGB image

In [ ]:
import os
import sys
import json
import glob
import datetime
import hashlib
import rasterio
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

sys.path.insert(0, '../')
from managers import managers

%load_ext autoreload
%autoreload 2

In [ ]:
# load without resetting
proj = managers.LandsatProject(
    project_root='../projects/test-landsat',
    reset=False)

In [ ]:
proj.stack(proj.derived_dataset, bands=[7,5,2])

In [ ]:
# future features
operations = proj.find_operations(method='stack', bands=[4,3,2])

proj.show_descendents(operation)
proj.show_ancestors(operation)

im = proj.open_dataset(operation='last')
im = proj.open_dataset(operation=operations[ind])

In [ ]:
proj.autogain(source='last', percentile=99)
proj.autogain(source='root', percentile=99)
proj.autogain(source=operations[0])

In [ ]:
source = proj.props['operations'][-1]['output']
output = source.replace('.TIF', '_AUTOGAIN.TIF')

In [ ]:
proj.props

In [ ]:
proj.new_dataset('tif', method='stack').path

In [ ]:
each_band = True
percentile = 99

def _autogain(im, percentile):
    
    minn, maxx = np.percentile(im[:], [100 - percentile, percentile])
    
    im -= minn
    im[im < 0] = 0
    im /= (maxx - minn)
    im[im > 1] = 1
    return im


with rasterio.open(source) as src:
    
    dst_profile = src.profile
    dst_profile['dtype'] = 'uint8'

    with rasterio.open(output, 'w', **dst_profile) as dst:

        if each_band:
            im_dst = np.zeros((len(src.indexes),) + src.shape)
            for ind, band in enumerate(src.indexes):
                im = src.read(band)
                im = im.astype('float64')
                im_dst[ind, :, :] = _autogain(im, percentile)
        else:
            im = src.read()
            im = im.astype('float64')
            im_dst = _autogain(im, percentile)

        im_dst *= 255
        im_dst = im_dst.astype('uint8')
        dst.write(im_dst)

In [ ]:
plt.imshow(np.stack((im_dst[0, :, :], im_dst[1, :, :], im_dst[2, :, :]), axis=2))